In [2]:
from keys import client_id, client_secret, user_name
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2
import random
from functools import reduce

In [3]:
scope = 'user-library-read playlist-modify-public playlist-read-private user-read-recently-played app-remote-control user-top-read'

redirect_uri = 'https://developer.spotify.com/dashboard/applications/0743a195f7654b5ab95560a95e89316a'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(user_name, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("NO TOKEN FOUND")


In [4]:
user_top_tracks = sp.current_user_top_tracks(limit=50, offset=0, time_range='long_term')
song_data = user_top_tracks["items"]

song_ids = []
song_names = []

for i in range(0, len(song_data)):
    if song_data[i]['id'] != None:
        song_ids.append(song_data[i]['id'])
        song_names.append(song_data[i]['name'])

song_features = []
for i in range(0, len(song_ids)):
    features = sp.audio_features(song_ids[i])
    for song in features:
        song_features.append(song)

top_tracks_df = pd.DataFrame(song_features, index=song_names)
top_tracks_df = top_tracks_df[["id", "acousticness", "danceability", "duration_ms", 
                         "energy", "instrumentalness",  "key", "liveness",
                         "loudness", "mode", "speechiness", "tempo", "valence"]]
top_tracks_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
HOLIDAY,6zFMeegAMYQo0mt8rXtrli,0.12000,0.810,154998,0.511,0.000000,5,0.0832,-6.924,0,0.1640,151.947,0.837
Death of a Bachelor,1BECwm5qkaBwlbfo4kpYx8,0.01370,0.462,203507,0.538,0.000000,0,0.4290,-5.527,1,0.0590,139.256,0.405
Daylight,6Ed1q0X8oSKSm4IIhiQbYg,0.00488,0.528,163906,0.749,0.000096,7,0.0949,-7.571,1,0.0479,163.944,0.729
Knock Knock,3uYm4MtU6jUQft2DtGqEoZ,0.03530,0.667,202807,0.922,0.000000,4,0.3640,-4.857,0,0.1280,108.952,0.509
Don't Matter,7I6DceMT3utDOHjcYCbrr4,0.24400,0.798,293053,0.443,0.000000,2,0.3260,-6.062,1,0.0403,125.232,0.354


In [5]:
top_tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, HOLIDAY to Team
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                50 non-null     object 
 1   acousticness      50 non-null     float64
 2   danceability      50 non-null     float64
 3   duration_ms       50 non-null     int64  
 4   energy            50 non-null     float64
 5   instrumentalness  50 non-null     float64
 6   key               50 non-null     int64  
 7   liveness          50 non-null     float64
 8   loudness          50 non-null     float64
 9   mode              50 non-null     int64  
 10  speechiness       50 non-null     float64
 11  tempo             50 non-null     float64
 12  valence           50 non-null     float64
dtypes: float64(9), int64(3), object(1)
memory usage: 5.5+ KB


In [6]:
featured_playlists = sp.featured_playlists(limit=50)
# Creating data frame for playlists
id = []
name = []
num_tracks = []
items = featured_playlists['playlists']['items']
for item in items:
    id.append(item["id"])
    name.append(item["name"])
    num_tracks.append(item["tracks"]["total"])

df_pl_data = pd.DataFrame({"id":id, "name": name, "num_tracks": num_tracks})

In [7]:
df_pl_data

,id,name,num_tracks
0,37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,50
1,37i9dQZF1DX4bSrsRWE9cd,Bliss,75
2,37i9dQZF1DWU3bkMPOyjie,Summer Rock Vibes,73
3,37i9dQZF1DXbm6HfkbMtFZ,Feel Good Dinner,115
4,37i9dQZF1DX4UtSsGT1Sbe,All Out 80s,150
5,37i9dQZF1DX186v583rmzp,I Love My '90s Hip-Hop,100
6,37i9dQZF1DXdPec7aLTmlC,Happy Hits!,100
7,37i9dQZF1DWY4xHQp97fN6,Get Turnt,100
8,37i9dQZF1DX4WYpdgoIcn6,Chill Hits,130
9,37i9dQZF1DX5NsFgylu4qQ,Waves,81


In [8]:
# Returns Individual Tracks In Playlists
def get_playlist_tracks(sp, playlist_id):
    data_track = sp.playlist_tracks(playlist_id, fields=None, limit=100, offset=0, market=None)['items']
    track_id = []
    track_name = []
    
    for data in data_track:
        track_id.append(data['track']['id'])
        track_name.append(data['track']['name'])

    return pd.DataFrame({"track_id":track_id, "track_name": track_name})

In [9]:
get_playlist_tracks(sp, '37i9dQZF1DWZKuerrwoAGz').head()

,track_id,track_name
0,6FE2iI43OZnszFLuLtvvmg,Classic
1,3DmW6y7wTEYHJZlLo1r6XJ,Shower
2,4E5P1XyAFtrjpiIxkydly4,Replay
3,3E7dfMvvCLUddWissuqMwr,Party In The U.S.A.
4,1CQ2cMfrmFM1YdfmjENKVE,She Looks So Perfect


In [10]:
# Function for getting audio features of individual tracks
def get_audio_features(sp, playlist_id):
    playlist = get_playlist_tracks(sp, playlist_id)
    audio_features = []
    for i in range(len(playlist)):
        song_data_id = playlist['track_id'][i]
        audio_features.append(sp.audio_features(song_data_id))
    features_array = []
    for features in audio_features:
        features = features[0]
        features_array.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness']
                             ])
    df_audio_features = pd.DataFrame(features_array, columns=['danceability', 'acousticness', 'energy', 'tempo', 'instrumentalness', 'loudness', 'liveness', 'duration_ms', 'key', 'valence', 'speechiness'])
    # DataFrame with audio features and playlist id/name
    df_playlist_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_features

In [11]:
get_audio_features(sp, '37i9dQZF1DXcRXFNfZr7Tp').head()

,track_id,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness
track_name,,,,,,,,,,,,
Beautiful Mistakes (feat. Megan Thee Stallion),6fRxMU4LWwyaSSowV441IU,0.713,0.0377,0.676,99.048,0.000000,-5.483,0.154,227395,10,0.721,0.0270
Peaches (feat. Daniel Caesar & Giveon),4iJyoBOLtHqaGxP12qzhQI,0.677,0.3210,0.696,90.030,0.000000,-6.181,0.420,198082,0,0.464,0.1190
deja vu,61KpQadow081I2AsbeLcsb,0.439,0.5930,0.610,181.088,0.000011,-7.236,0.341,215508,9,0.172,0.1160
BED,7jXQUrVhEpXdymfFWNDnQW,0.663,0.0134,0.783,123.986,0.001790,-4.585,0.325,178088,6,0.622,0.0393
Met Him Last Night (feat. Ariana Grande),0BI0hfbmqybnd3TezrDME3,0.538,0.2200,0.512,144.978,0.000000,-4.548,0.101,204632,4,0.120,0.0262


In [13]:
# Finding mean of audio features for each playlist
def audio_features_mean(sp, playlist_id):
    Playlist = get_audio_features(sp, playlist_id)
    return pd.DataFrame(Playlist.mean(), columns= [playlist_id])

In [30]:
audio_features_mean(sp, '37i9dQZF1DXbYM3nMM0oPk')

KeyboardInterrupt: 

In [29]:
# Merges average of audio features of each playlist into a single DataFrame
primary_df = []
for i in range(len(df_pl_data)):
    primary_df.append(audio_features_mean(sp, df_pl_data['id'][i]))

In [16]:
primary_df

[                  37i9dQZF1DXcBWIGoYBM5M
 danceability                    0.686820
 acousticness                    0.255789
 energy                          0.616880
 tempo                         119.491220
 instrumentalness                0.005073
 loudness                       -6.547720
 liveness                        0.184132
 duration_ms                184768.500000
 key                             5.180000
 valence                         0.464922
 speechiness                     0.081396,
                   37i9dQZF1DX4bSrsRWE9cd
 danceability                    0.493147
 acousticness                    0.816849
 energy                          0.235191
 tempo                         118.281827
 instrumentalness                0.162121
 loudness                      -14.750040
 liveness                        0.132576
 duration_ms                231474.000000
 key                             3.813333
 valence                         0.249524
 speechiness                     

In [17]:
# Turns playlist feature mean data into correctly formatted DataFrame
X_data = reduce(lambda left,right: pd.merge(left,right, left_index=True, right_index=True), primary_df)
X_data

,37i9dQZF1DXcBWIGoYBM5M,37i9dQZF1DX4bSrsRWE9cd,37i9dQZF1DWU3bkMPOyjie,37i9dQZF1DXbm6HfkbMtFZ,37i9dQZF1DX4UtSsGT1Sbe,37i9dQZF1DX186v583rmzp,37i9dQZF1DXdPec7aLTmlC,37i9dQZF1DWY4xHQp97fN6,37i9dQZF1DX4WYpdgoIcn6,37i9dQZF1DX5NsFgylu4qQ,37i9dQZF1DX82pCGH5USnM,37i9dQZF1DWYs83FtTMQFw
danceability,0.686820,0.493147,0.507767,0.727840,0.692520,0.767620,0.682550,0.806070,0.650250,0.662210,0.718130,0.759273
acousticness,0.255789,0.816849,0.032725,0.251532,0.174843,0.113683,0.134390,0.115721,0.356874,0.307635,0.259253,0.199679
energy,0.616880,0.235191,0.883452,0.610740,0.723460,0.684230,0.705310,0.621680,0.525280,0.554148,0.558350,0.581523
tempo,119.491220,118.281827,137.531342,108.492510,120.156330,104.043510,119.150000,128.961100,116.588920,111.621988,114.381740,118.657102
instrumentalness,0.005073,0.162121,0.036276,0.078397,0.026667,0.004795,0.005998,0.006484,0.005211,0.032039,0.841477,0.006970
loudness,-6.547720,-14.750040,-4.348123,-7.529030,-8.647150,-7.242940,-5.431520,-6.459370,-7.176000,-8.310815,-12.104650,-6.947659
liveness,0.184132,0.132576,0.217848,0.187282,0.185297,0.228368,0.180783,0.163975,0.172499,0.167702,0.128697,0.168328
duration_ms,184768.500000,231474.000000,212175.178082,222785.550000,247861.220000,259391.130000,182980.350000,182556.170000,186238.160000,213390.765432,218269.740000,177628.102273
key,5.180000,3.813333,5.191781,5.270000,4.620000,6.220000,5.330000,4.640000,4.990000,5.827160,5.410000,5.045455
valence,0.464922,0.249524,0.602192,0.660040,0.740530,0.660430,0.573460,0.497988,0.444416,0.369707,0.300440,0.533468


In [18]:
# Turns my favorite track data (Y_data) into a DataFrame with audio features averaged
# maybe change mean to median
Y_data = pd.DataFrame(top_tracks_df.mean(), columns=['top_tracks']).drop('mode')
Y_data

,top_tracks
acousticness,0.128677
danceability,0.646100
duration_ms,212794.200000
energy,0.700260
instrumentalness,0.000695
key,5.660000
liveness,0.197508
loudness,-5.582480
speechiness,0.095180
tempo,120.883060


In [19]:
# Model for predicting playlist of "best-fit" using feature_importances_
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=44, max_depth=5, max_features=8, oob_score=True)
model.fit(X_data, Y_data['top_tracks'])
pl_rank = model.feature_importances_
pl_importances = pd.DataFrame(pl_rank, index = X_data.columns, columns=['importance']).sort_values('importance',                                        ascending=False)

pl_importances

,importance
37i9dQZF1DWU3bkMPOyjie,0.168674
37i9dQZF1DX82pCGH5USnM,0.116964
37i9dQZF1DX4UtSsGT1Sbe,0.111874
37i9dQZF1DXdPec7aLTmlC,0.111521
37i9dQZF1DX5NsFgylu4qQ,0.075277
37i9dQZF1DWYs83FtTMQFw,0.069473
37i9dQZF1DX4bSrsRWE9cd,0.069058
37i9dQZF1DXbm6HfkbMtFZ,0.063317
37i9dQZF1DXcBWIGoYBM5M,0.060841
37i9dQZF1DWY4xHQp97fN6,0.055801


In [20]:
pl_t3 = pl_importances.index[0:3]
frames = []
for i in range(len(pl_t3)):
    frame = get_audio_features(sp, pl_t3[i])
    frames.append(frame)
    
recommended_pl = pd.concat(frames)

In [21]:
recommended_pl.head()

,track_id,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness
track_name,,,,,,,,,,,,
Dani California,10Nmj3JCNoMeBQ87uw5j8k,0.556,0.019300,0.913,96.184,0.000009,-2.360,0.346,282160,0,0.730,0.0437
Holiday,5vfjUAhefN7IjHbTvVCT4Z,0.538,0.000138,0.884,146.511,0.000000,-3.251,0.466,233027,8,0.736,0.0310
When You Were Young,70wYA8oYHoMzhRRkARoMhU,0.467,0.000152,0.988,130.433,0.048400,-3.313,0.280,220427,11,0.321,0.1120
Black Betty - Edit,7uSsHbBFFAnkRQR1rDwP3L,0.562,0.000021,0.865,124.047,0.591000,-6.476,0.219,205973,7,0.350,0.0922
Are You Gonna Be My Girl,305WCRhhS10XUcH6AEwZk6,0.613,0.001480,0.953,105.046,0.000582,-3.435,0.152,213800,2,0.537,0.0855


In [22]:
pl_rec = recommended_pl.set_index('track_id')
pl_rec.sort_index( axis=1, level=None, ascending=True, inplace=True, kind='quicksort')

pl_rec.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
track_id,,,,,,,,,,,
10Nmj3JCNoMeBQ87uw5j8k,0.019300,0.556,282160,0.913,0.000009,0,0.346,-2.360,0.0437,96.184,0.730
5vfjUAhefN7IjHbTvVCT4Z,0.000138,0.538,233027,0.884,0.000000,8,0.466,-3.251,0.0310,146.511,0.736
70wYA8oYHoMzhRRkARoMhU,0.000152,0.467,220427,0.988,0.048400,11,0.280,-3.313,0.1120,130.433,0.321
7uSsHbBFFAnkRQR1rDwP3L,0.000021,0.562,205973,0.865,0.591000,7,0.219,-6.476,0.0922,124.047,0.350
305WCRhhS10XUcH6AEwZk6,0.001480,0.613,213800,0.953,0.000582,2,0.152,-3.435,0.0855,105.046,0.537


In [23]:
Y_data_sorted = Y_data.sort_index( axis=0, level=None, ascending=True, inplace=False, kind='quicksort').squeeze(1)

Y_data_sorted.shape

(11,)

In [24]:
Y_data_sorted

acousticness             0.128677
danceability             0.646100
duration_ms         212794.200000
energy                   0.700260
instrumentalness         0.000695
key                      5.660000
liveness                 0.197508
loudness                -5.582480
speechiness              0.095180
tempo                  120.883060
valence                  0.539028
Name: top_tracks, dtype: float64

In [25]:
df_main_var = pl_rec.subtract(Y_data_sorted, axis=1)
df_main_var = df_main_var.divide(Y_data_sorted, axis=1) 

df_main_var.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
track_id,,,,,,,,,,,
10Nmj3JCNoMeBQ87uw5j8k,-0.850012,-0.139452,0.325976,0.303801,-0.987637,-1.000000,0.751828,-0.577249,-0.540870,-0.204322,0.354290
5vfjUAhefN7IjHbTvVCT4Z,-0.998928,-0.167312,0.095082,0.262388,-1.000000,0.413428,1.359398,-0.417642,-0.674301,0.212006,0.365421
70wYA8oYHoMzhRRkARoMhU,-0.998819,-0.277202,0.035869,0.410905,68.657046,0.943463,0.417664,-0.406536,0.176718,0.079001,-0.404484
7uSsHbBFFAnkRQR1rDwP3L,-0.999840,-0.130166,-0.032055,0.235255,849.564337,0.236749,0.108816,0.160058,-0.031309,0.026174,-0.350683
305WCRhhS10XUcH6AEwZk6,-0.988498,-0.051230,0.004727,0.360923,-0.162388,-0.646643,-0.230411,-0.384682,-0.101702,-0.131011,-0.003762


In [26]:
df_main_var['variation'] = df_main_var.sum(axis=1)

df_main_var['variation'] = df_main_var['variation'].abs()

In [27]:
df_main_var

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,variation
track_id,,,,,,,,,,,,
10Nmj3JCNoMeBQ87uw5j8k,-0.850012,-0.139452,0.325976,0.303801,-0.987637,-1.000000,0.751828,-0.577249,-0.540870,-0.204322,0.354290,2.563647
5vfjUAhefN7IjHbTvVCT4Z,-0.998928,-0.167312,0.095082,0.262388,-1.000000,0.413428,1.359398,-0.417642,-0.674301,0.212006,0.365421,0.550461
70wYA8oYHoMzhRRkARoMhU,-0.998819,-0.277202,0.035869,0.410905,68.657046,0.943463,0.417664,-0.406536,0.176718,0.079001,-0.404484,68.633626
7uSsHbBFFAnkRQR1rDwP3L,-0.999840,-0.130166,-0.032055,0.235255,849.564337,0.236749,0.108816,0.160058,-0.031309,0.026174,-0.350683,848.787336
305WCRhhS10XUcH6AEwZk6,-0.988498,-0.051230,0.004727,0.360923,-0.162388,-0.646643,-0.230411,-0.384682,-0.101702,-0.131011,-0.003762,2.334679
...,...,...,...,...,...,...,...,...,...,...,...,...
0bVtevEgtDIeRjCJbK3Lmv,-0.817372,-0.308157,0.278357,0.362351,578.995648,0.413428,0.508800,0.445415,-0.210969,0.021260,-0.385932,579.302830
7woZGXjtaYwuvSoU1zToB3,0.383314,-0.045039,0.206175,-0.378802,-1.000000,-0.116608,-0.583814,1.426699,-0.731036,-0.301953,0.114970,1.026096
5yGTQzYbEdY6B9RFZJypgt,-0.402377,0.094258,0.076660,0.083883,-0.959127,0.943463,-0.607611,1.208338,-0.516705,-0.047493,0.771708,0.644996


In [28]:
# Picking 30 least variated songs from df_main_var
rec_songs = df_main_var.nsmallest(30,'variation')

rec_songs

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,variation
track_id,,,,,,,,,,,,
21qnJAMtzC6S5SESuqQLEK,-0.885760,0.369757,0.505243,0.205267,-0.993538,0.766784,-0.749884,0.652491,-0.229880,0.030765,0.326462,0.002293
2D4D3hiOf5U0W6SvJoCQph,-0.320778,0.030800,0.336376,0.299517,-0.987896,-0.293286,0.701197,0.156654,-0.580794,-0.030964,0.704921,0.015747
3SFXsFpeGmBTtQvKiwYMDA,-0.316115,0.119022,-0.113383,0.278097,-1.000000,0.943463,-0.636977,-0.335958,0.071654,0.186047,0.786549,0.017601
1A2PWRltFrX8iB8IP3CUgo,0.165714,-0.052778,0.183303,-0.143175,-0.995625,0.590106,0.124005,0.960598,-0.656440,-0.080078,-0.126205,0.030576
2tUBqZG2AbRi7Q0BIrVrEj,0.608685,0.097353,0.368895,0.176706,-0.558167,-0.823322,-0.550398,0.580659,-0.524060,-0.017083,0.608451,0.032280
2WfaOiMkCvy7F5fcp2zZ8L,-0.860114,-0.113140,0.058675,0.288093,0.798994,0.060071,-0.530146,0.368209,-0.432654,-0.301705,0.625147,0.038570
2H7PHVdQ3mXqEHXcvclTB0,0.064685,0.340350,0.782318,0.042470,-1.000000,-0.116608,-0.573182,0.469060,-0.194158,-0.019523,0.159494,0.045094
4yKZACkuudvfd600H2dQie,0.593142,0.360471,0.000530,-0.106046,-0.912641,0.236749,-0.716467,0.136412,-0.365413,-0.082882,0.795825,0.060320
0snPJPxkk0MbTc0xeUvAPt,-0.067429,0.080328,0.272215,0.105304,-0.996747,-1.000000,0.994856,0.548057,-0.495692,0.101867,0.356145,0.101097
